# On-Policy First-Visit MC Control (for $\epsilon$-soft policies), estimates $\pi\approx\pi_*$

In [15]:
import gymnasium as gym
import numpy as np
from tqdm import tqdm
import time

In [16]:
SEED = 42
np.random.seed(SEED)

In [17]:
def make_greedy_policy(Q):
    def policy(state):
        return np.argmax(Q[state])
    return policy

In [18]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy(state):
        action_probs = np.ones(nA) * (epsilon / nA)
        best_action = np.argmax(Q[state])
        action_probs[best_action] += (1.0 - epsilon)
        return action_probs
    return policy

In [19]:
def generate_episode(env, policy, max_steps=100):
    episode = []
    state = env.reset()[0]
    done = False
    steps = 0

    while not done and steps < max_steps:
        action_probs = policy(state)
        action = np.random.choice(len(action_probs), p=action_probs)
        next_state, reward, done, _, _ = env.step(action)

        if not done:
            reward -= 0.01  # Step penalty
        elif reward == 0:
            reward = -1.0   # Hole penalty
        else:
            reward = 5.0    # Reaching goal

        episode.append((state, action, reward))
        state = next_state
        steps += 1

    return episode

In [20]:
def on_policy_first_visit_mc_control(env, epsilon_start, num_episodes, gamma):
    nS = env.observation_space.n
    nA = env.action_space.n
    Q = np.zeros((nS, nA))
    returns_count = np.zeros((nS, nA))

    epsilon_min = 0.05
    epsilon_decay = 0.9993  # Slower decay keeps more exploration

    for i in tqdm(range(num_episodes), desc="Training episodes"):
        epsilon = max(epsilon_min, epsilon_start * (epsilon_decay ** i))
        policy = make_epsilon_greedy_policy(Q, epsilon, nA)
        episode = generate_episode(env, policy)

        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if (state, action) not in visited:
                visited.add((state, action))
                returns_count[state][action] += 1
                Q[state][action] += (1 / returns_count[state][action]) * (G - Q[state][action])

    return make_greedy_policy(Q), Q

In [21]:
custom_map = [
    "SFFHF",
    "HFHFF",
    "FFFHF",
    "FHFFF",
    "HFFFG"
]

# Hyperparameters
n_episodes = 3000
gamma = 0.98
epsilon_start = 1.0

In [22]:
# Train environment
env = gym.make("FrozenLake-v1", desc=custom_map, is_slippery=False)
policy, Q = on_policy_first_visit_mc_control(env, epsilon_start, n_episodes, gamma)

Training episodes: 100%|██████████| 3000/3000 [00:02<00:00, 1450.48it/s]


In [23]:
# Save Q-table for later use
np.savez('results/FrozenLake.npz', Q=Q)

In [24]:
env = gym.make("FrozenLake-v1", desc=custom_map, is_slippery=False, render_mode="human")
state = env.reset()[0] 
done = False

while not done:
    env.render()
    action = policy(state)
    state, reward, done, truncate, info = env.step(action)
    time.sleep(0.3)
env.close()